# Install Missing Packages on Colab

In [1]:
!pip install xmltodict

# Import Packages

In [2]:
import pandas as pd
import re

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import xmltodict
import pandas as pd
from tqdm.notebook import tqdm

# Define a set of tools to interact with NERC API
This is just preliminary and lots of space for improvement an refinement. 
A specific python package within the [CIOOS metadata standard repository ](https://github.com/cioos-siooc/cioos-siooc-standards) would be ideal.

In [3]:
def get_nvs_variable_info(id=None,
                          variable=None,
                          vocabulary=None,
                          nvs_url="http://vocab.nerc.ac.uk/collection/",
                          version="current",
                          format_output="?_profile=skos&_mediatype=application/ld+json"
                          ):
    """
    Method to parse the json format from the NERC NVS servers
    """
    if id:
        url = id
    else:
        # Define the base of the URL
        url = nvs_url + '/' + vocabulary + '/' + version

        # Add the optional variable
        if variable:
            url = url + '/' + variable

    # Get the response from the NERC servers
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url + '/' + format_output)
    return response.json()


def _get_language(item_list,language='en'):
    if type(language) is str:
        return [item['@value'] for item in item_list if item['@language']==language][0]
    elif type(language) is list:
        return {item['@language']: item['@value'] \
                for item in item_list if item['@language'] in language}

def _get_section(vocabulary,section):
    for name, item in vocabulary.items():
        if name.endswith(section):
            if '@language' in item[0]:
                return _get_language(item)
            elif '@value' in item[0] and len(item)==1:
                return item[0]['@value']
            else:
                return item

def get_nerc_id_info(id,
                     only=None,
                     vocabularies=None,
                     language='en'):
    if vocabularies is None:
        vocabularies  = ['P02','P06','P07']
    
    # Get all the info for this id
    id_info = get_nvs_variable_info(id)[0]
    
    # If only one specific is request just give that.
    if only:
        return [_get_section(id_info,item) for item in only]
        
    relationships = ("broader","related","narrower")
    # Initialize dictionary
    output = {'@id':id}
    for vocab in vocabularies:
        for relationship in relationships:
            output[vocab+'_'+relationship+'_id'] = []


    for key, value in id_info.items():

        if key.endswith('prefLabel'):
            output['prefLabel'] = value[0]['@value']
        
        if key.endswith('altLabel'):
            output['altLabel'] = value[0]['@value']

        if key.endswith('definition'):
            # Get English Definition
            output['definition-en'] = _get_language(value)           

        if key.endswith(relationships):
            relationship = key.rsplit('#')[1]

            for item in id_info[key]:
                for vocab in vocabularies:
                    if '/collection/'+vocab+'/' in item['@id']:
                        output[vocab+'_'+relationship+'_id']  += [item['@id']]
    return output


# Retrieve all the different vocabularies needed
This only retrieve for each vocabularies, the id and the prefered Label.

In [53]:
# Vocabularies of interest
vocab_of_interest = ['P01','P02','P06','P07','P04','P64']
# Extract the PrefLabel
def retrieve_value(line):
    if len(line)==1 and '@value' in line[0]:
        return line[0]['@value']
    else:
        return None

# Retrieve All Vocabularies of interest
df_vocab = pd.DataFrame()
pbar = tqdm(vocab_of_interest)
for vocab in pbar:
    pbar.set_description("Retrieve %s" % vocab)
    response = get_nvs_variable_info(vocabulary=vocab)
    df_vocab = df_vocab.append(pd.DataFrame(response))

df_vocab['prefLabel'] = df_vocab['http://www.w3.org/2004/02/skos/core#prefLabel'].apply(retrieve_value)


In [54]:
df_vocab

,@id,http://www.w3.org/2004/02/skos/core#prefLabel,@type,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#member,prefLabel
0,http://vocab.nerc.ac.uk/collection/P01/current...,"[{'@value': 'Concentration of 2,3,4,4',5-penta...",NaN,NaN,NaN,"Concentration of 2,3,4,4',5-pentachlorobipheny..."
1,http://vocab.nerc.ac.uk/collection/P01/current...,[{'@value': 'Absorbance of electromagnetic rad...,NaN,NaN,NaN,Absorbance of electromagnetic radiation (410nm...
2,http://vocab.nerc.ac.uk/collection/P01/current...,"[{'@value': 'Concentration of 2,2',4,5,5'-pent...",NaN,NaN,NaN,"Concentration of 2,2',4,5,5'-pentachlorobiphen..."
3,http://vocab.nerc.ac.uk/collection/P01/current...,"[{'@value': 'Concentration of 4,4'-dichlorodip...",NaN,NaN,NaN,"Concentration of 4,4'-dichlorodiphenyltrichlor..."
4,http://vocab.nerc.ac.uk/collection/P01/current...,[{'@value': 'Lipid-normalised concentration of...,NaN,NaN,NaN,"Lipid-normalised concentration of 2,2',3,4,5'-..."
...,...,...,...,...,...,...
1542,http://vocab.nerc.ac.uk/collection/P64/current...,[{'@value': 'EARTH SCIENCE > Terrestrial Hydro...,NaN,NaN,NaN,EARTH SCIENCE > Terrestrial Hydrosphere > Wate...
1543,http://vocab.nerc.ac.uk/collection/P64/current...,[{'@value': 'EARTH SCIENCE > Atmosphere > Air ...,NaN,NaN,NaN,EARTH SCIENCE > Atmosphere > Air Quality > Emi...
1544,http://vocab.nerc.ac.uk/collection/P64/current...,[{'@value': 'EARTH SCIENCE > Biosphere > Ecolo...,NaN,NaN,NaN,EARTH SCIENCE > Biosphere > Ecological Dynamic...
1545,http://vocab.nerc.ac.uk/collection/P64/current...,[{'@value': 'EARTH SCIENCE > Solid Earth > Roc...,NaN,NaN,NaN,EARTH SCIENCE > Solid Earth > Rocks/Minerals >...


# Define BODC (P01) vocabulary terms of interest

In [55]:
# Filter the data to retrieve only specific terms 
#  Those terms can easily be matched to CF terms
search_terms = {
    "Pressure (spatial coordinate) exerted by the water body": {
        "P07": "sea_water_pressure"
    },
    "Depth (spatial coordinate) relative to water surface in the water body": {
        "P07": "depth"
    },
    "Surface elevation.*of the water body": {
        "P07": "sea_floor_depth_below_sea_surface"
    },
    "Temperature.*of the water body": {
        "P07": ["sea_water_temperature", "sea_surface_temperature"]
    },
    "Electrical conductivity.*of the water body": {
        "P07": "sea_water_electrical_conductivity"
    },
    "Salinity.*of the water body": {
        "P07": "sea_water_salinity"
    },
    "Practical salinity.*of the water body": {
        "P07": "sea_water_practical_salinity"
    },
    "Concentration of oxygen {O2 CAS 7782-44-7} per unit mass of the water body": {
        "P07": ["moles_of_oxygen_per_unit_mass_in_sea_water"]
    },
    "Concentration of oxygen {O2 CAS 7782-44-7} per unit volume of the water body": {
        "P07": [
            "volume_fraction_of_oxygen_in_sea_water",
            "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
            "mass_concentration_of_oxygen_in_sea_water"
        ]
    },
    "Concentration of phosphate {PO43- CAS 14265-44-2} per unit mass of the water body": {
        "P07": "moles_of_phosphate_per_unit_mass_in_sea_water"
    },
    "Concentration of phosphate {PO43- CAS 14265-44-2} per unit volume of the water body": {
        "P07": [
            "mass_concentration_of_phosphate_in_sea_water",
            "mole_concentration_of_phosphate_in_sea_water"
        ]
    },
    "Concentration of silicate {SiO44- CAS 17181-37-2} per unit mass of the water body": {
        "P07": "moles_of_silicate_per_unit_mass_in_sea_water"
    },
    "Concentration of silicate {SiO44- CAS 17181-37-2} per unit volume of the water body": {
        "P07": [
            "mass_concentration_of_silicate_in_sea_water",
            "mole_concentration_of_silicate_in_sea_water"
        ]
    },
    "Concentration of nitrite {NO2- CAS 14797-65-0} per unit volume of the water body": {
        "P07": "moles_of_nitrite_per_unit_mass_in_sea_water"
    },
    "Concentration of nitrite {NO2- CAS 14797-65-0} per unit mass of the water body": {
        "P07": "mole_concentration_of_nitrite_in_sea_water"
    },
    "Concentration of nitrate+nitrite {NO3+NO2} per unit volume of the water body": {
        "P07": "mole_concentration_of_nitrate_and_nitrite_in_sea_water"
    },
    "Concentration of nitrate+nitrite {NO3+NO2} per unit mass of the water body": {
        "P07": "moles_of_nitrate_and_nitrite_per_unit_mass_in_sea_water"
    },
    "Concentration of chlorophyll-a {chl-a CAS 479-61-8} per unit volume of the water body": {
        "P07": "mass_concentration_of_chlorophyll_a_in_sea_water"
    }
}
list(search_terms.keys())

['Pressure (spatial coordinate) exerted by the water body',
 'Depth (spatial coordinate) relative to water surface in the water body',
 'Surface elevation.*of the water body',
 'Temperature.*of the water body',
 'Electrical conductivity.*of the water body',
 'Salinity.*of the water body',
 'Practical salinity.*of the water body',
 'Concentration of oxygen {O2 CAS 7782-44-7} per unit mass of the water body',
 'Concentration of oxygen {O2 CAS 7782-44-7} per unit volume of the water body',
 'Concentration of phosphate {PO43- CAS 14265-44-2} per unit mass of the water body',
 'Concentration of phosphate {PO43- CAS 14265-44-2} per unit volume of the water body',
 'Concentration of silicate {SiO44- CAS 17181-37-2} per unit mass of the water body',
 'Concentration of silicate {SiO44- CAS 17181-37-2} per unit volume of the water body',
 'Concentration of nitrite {NO2- CAS 14797-65-0} per unit volume of the water body',
 'Concentration of nitrite {NO2- CAS 14797-65-0} per unit mass of the water

In [56]:
# Filter terms retrieved for exclusively the P01 (BODC) terms matching the list above
is_matching = df_vocab['prefLabel'].str.contains('|'.join(search_terms.keys()), regex=True) & \
    df_vocab['@id'].str.contains("collection\/P01", regex=True)

df_p01 = df_vocab.loc[is_matching][['@id','prefLabel']].sort_values('prefLabel')

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# Parse BODC prefLabels to retrieve specific information

In [57]:
# Retrieve information from P01 terms 
# Parse vocabulary, version, term and index value from term id:
df_p01[['Vocabulary','version','term','index']] = \
    df_p01['@id'].str.extract('collection/(.*?)/(.*?)/([A-Z]*?)([0-9]*?)/')

# Drop repeated terms that have different number at the end (ex: [TEMP01, TEMP02] -> [TEMP01]) 
df_p01 = df_p01.sort_values(['term','index']).drop_duplicates('term')

# From P01 prefLabel, retrieve some of the information
df_p01['instrument-method'] = df_p01['prefLabel'].str.extract('by (.*?)(?=$| and)')
df_p01['calibration'] = df_p01['prefLabel'].str.extract('and ((?!.*and).*(calibration|verification).*?)(?=$| and)')[0]
df_p01['scale'] = df_p01['prefLabel'].str.extract('(ITS-90|IPTS-68|IPTS-48|PSS-78|Practical.*salinity?)',
                                          flags=re.IGNORECASE).replace
df_p01['scale'] = df_p01['scale'].str.replace('Practical.*salinity','PSS-78',1).dropna()
df_p01['size'] = df_p01['prefLabel'].str.extract('\[(.*?)\]')
df_p01['reference'] = df_p01['prefLabel'].str.extract('\((.*(Level|datum).*?)\)',re.IGNORECASE)[0]
df_p01

,@id,prefLabel,Vocabulary,version,term,index,instrument-method,calibration,scale,size,reference
34990,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVACDR,,radar water level recorder,NaN,NaN,NaN,UK Admiralty Chart Datum
26200,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVADPH,,fixed half-tide pressure sensor,NaN,NaN,NaN,UK Admiralty Chart Datum
12270,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBG,01,bubbler tide gauge,NaN,NaN,NaN,unspecified datum
39943,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBH,01,bubbler tide gauge half-tide sensor,NaN,NaN,NaN,unspecified datum
26422,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (Chart Datum) of the water b...,P01,current,ASLVCDPG,,fixed in-situ pressure sensor,NaN,NaN,NaN,Chart Datum
...,...,...,...,...,...,...,...,...,...,...,...
36696,http://vocab.nerc.ac.uk/collection/P01/current...,Temperature of the water body by in-situ therm...,P01,current,TEMPTC,01,in-situ thermistor,NaN,NaN,NaN,NaN
9933,http://vocab.nerc.ac.uk/collection/P01/current...,Temperature of the water body,P01,current,TEMPZZXX,,NaN,NaN,NaN,NaN,NaN
17427,http://vocab.nerc.ac.uk/collection/P01/current...,Temperature of the water body by in-situ therm...,P01,current,TMCLCCI,1,in-situ thermometer,calibration against independent measurements,NaN,NaN,NaN
12910,http://vocab.nerc.ac.uk/collection/P01/current...,Temperature of the water body by in-situ therm...,P01,current,TMCLCCL,1,in-situ thermometer,laboratory calibration applied,NaN,NaN,NaN


# Review each P01 terms connections

In [58]:
# For each of the terms available retrieve list of broader, equivalent and narrower 
#  terms available in different vocabularies
terms_more_info = []
print('Retrieve more detailed information from each term:')
pbar = tqdm(df_p01['@id'])
for id in pbar:
    terms_more_info+= [get_nerc_id_info(id, vocabularies=vocab_of_interest  )]

df_p01 = df_p01.merge(pd.DataFrame(terms_more_info), on='@id',suffixes=('','_x'))

Retrieve more detailed information from each term:


In [59]:
# Retrieve prefLabel from NERC id
df_vocab_indexed = df_vocab.set_index('@id')

def simplify_cell(cell):
    if len(cell)==0:
        return None
    elif len(cell)==1:
        return cell[0]
    else:
        return cell

# Retrieve for each ID listed the prefered Label
df_prefLabel = df_p01.filter(like='_id')\
    .applymap(lambda x: [df_vocab_indexed.loc[y]['prefLabel'] for y in x if y])\
    .add_suffix('_prefLabel')\
    .applymap(simplify_cell)

# Simplify cells to have either a single value, None, or a list of multiple terms exist.
df_p01[df_prefLabel.columns] = df_prefLabel
df_p01[df_p01.filter(regex='_id$').columns] = df_p01.filter(regex='_id$').applymap(simplify_cell)


In [60]:
# Show result
df_p01

,@id,prefLabel,Vocabulary,version,term,index,instrument-method,calibration,scale,size,reference,P01_broader_id,P01_related_id,P01_narrower_id,P02_broader_id,P02_related_id,P02_narrower_id,P06_broader_id,P06_related_id,P06_narrower_id,P07_broader_id,P07_related_id,P07_narrower_id,P04_broader_id,P04_related_id,P04_narrower_id,P64_broader_id,P64_related_id,P64_narrower_id,altLabel,definition-en,prefLabel_x,P01_broader_id_prefLabel,P01_related_id_prefLabel,P01_narrower_id_prefLabel,P02_broader_id_prefLabel,P02_related_id_prefLabel,P02_narrower_id_prefLabel,P06_broader_id_prefLabel,P06_related_id_prefLabel,P06_narrower_id_prefLabel,P07_broader_id_prefLabel,P07_related_id_prefLabel,P07_narrower_id_prefLabel,P04_broader_id_prefLabel,P04_related_id_prefLabel,P04_narrower_id_prefLabel,P64_broader_id_prefLabel,P64_related_id_prefLabel,P64_narrower_id_prefLabel
0,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVACDR,,radar water level recorder,NaN,NaN,NaN,UK Admiralty Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SL_UKACD_radar,Height of the sea surface relative to the low ...,Surface elevation (UK Admiralty Chart Datum) o...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None
1,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVADPH,,fixed half-tide pressure sensor,NaN,NaN,NaN,UK Admiralty Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,ACD_HTSL_press,Unavailable,Surface elevation (UK Admiralty Chart Datum) o...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None
2,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBG,01,bubbler tide gauge,NaN,NaN,NaN,unspecified datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SeaLvl_bubbler,Unavailable,Surface elevation (unspecified datum) of the w...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None
3,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBH,01,bubbler tide gauge half-tide sensor,NaN,NaN,NaN,unspecified datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,HTSeaLvl,Unavailable,Surface elevation (unspecified datum) of the w...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None
4,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (Chart Datum) of the water b...,P01,current,ASLVCDPG,,fixed in-situ pressure sensor,NaN,NaN,NaN,Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SL_CD_press,Unavailable,Surface elevation (Chart Datum) of the water b...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,http://vocab.nerc.ac.uk/collection/P01/current...,Temperature of the water body by in-situ therm...,P01,current,TEMPTC,01,in-situ thermistor,NaN,NaN,NaN,NaN,http://vocab.nerc.ac.uk/collection/P01/current...,N

In [119]:
# Add suggested P07
df_p01['suggested_P07'] = None
for index, row in df_p01.iterrows():
    for search_key, matching in search_terms.items(): 
        if re.search(search_key, row['prefLabel']) and  matching['P07']:
            if type(matching['P07']) is list:
                matching['P07'] = ','.join(matching['P07'])
            df_p01.loc[index,'suggested_P07'] = matching['P07']


In [120]:
df_p01

,@id,prefLabel,Vocabulary,version,term,index,instrument-method,calibration,scale,size,reference,P01_broader_id,P01_related_id,P01_narrower_id,P02_broader_id,P02_related_id,P02_narrower_id,P06_broader_id,P06_related_id,P06_narrower_id,P07_broader_id,P07_related_id,P07_narrower_id,P04_broader_id,P04_related_id,P04_narrower_id,P64_broader_id,P64_related_id,P64_narrower_id,altLabel,definition-en,prefLabel_x,P01_broader_id_prefLabel,P01_related_id_prefLabel,P01_narrower_id_prefLabel,P02_broader_id_prefLabel,P02_related_id_prefLabel,P02_narrower_id_prefLabel,P06_broader_id_prefLabel,P06_related_id_prefLabel,P06_narrower_id_prefLabel,P07_broader_id_prefLabel,P07_related_id_prefLabel,P07_narrower_id_prefLabel,P04_broader_id_prefLabel,P04_related_id_prefLabel,P04_narrower_id_prefLabel,P64_broader_id_prefLabel,P64_related_id_prefLabel,P64_narrower_id_prefLabel,suggested_P07
0,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVACDR,,radar water level recorder,NaN,NaN,NaN,UK Admiralty Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SL_UKACD_radar,Height of the sea surface relative to the low ...,Surface elevation (UK Admiralty Chart Datum) o...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None,sea_floor_depth_below_sea_surface
1,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (UK Admiralty Chart Datum) o...,P01,current,ASLVADPH,,fixed half-tide pressure sensor,NaN,NaN,NaN,UK Admiralty Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,ACD_HTSL_press,Unavailable,Surface elevation (UK Admiralty Chart Datum) o...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None,sea_floor_depth_below_sea_surface
2,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBG,01,bubbler tide gauge,NaN,NaN,NaN,unspecified datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SeaLvl_bubbler,Unavailable,Surface elevation (unspecified datum) of the w...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None,sea_floor_depth_below_sea_surface
3,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (unspecified datum) of the w...,P01,current,ASLVBH,01,bubbler tide gauge half-tide sensor,NaN,NaN,NaN,unspecified datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,HTSeaLvl,Unavailable,Surface elevation (unspecified datum) of the w...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None,sea_floor_depth_below_sea_surface
4,http://vocab.nerc.ac.uk/collection/P01/current...,Surface elevation (Chart Datum) of the water b...,P01,current,ASLVCDPG,,fixed in-situ pressure sensor,NaN,NaN,NaN,Chart Datum,None,None,None,http://vocab.nerc.ac.uk/collection/P02/current...,None,None,None,http://vocab.nerc.ac.uk/collection/P06/current...,None,None,None,None,None,None,None,None,None,None,SL_CD_press,Unavailable,Surface elevation (Chart Datum) of the water b...,None,None,None,Sea level,None,None,None,Metres,None,None,None,None,None,None,None,None,None,None,sea_floor_depth_below_sea_surface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,http://vocab.nerc.ac

# Save to a csv file

In [121]:
df_p01.sort_values('prefLabel').to_csv('NERC_P01_vocabulary.csv')